In [8]:
import requests
import pandas as pd
import numpy as np

#### Definitions ####

In [159]:
current_gameweek = requests.get(bootstrap_url).json()['events']['current']

owners = ['Luke','Emma','Fat','Olivia','Claire','Matt','George']
owner_ids = ['18900','18935','18967','19180','19424','20043','20626']
team_details_url = {}
lineup_url = {}

for owner_id, owner in enumerate(owners):
    team_details_url[owner] = 'https://draft.premierleague.com/api/entry/'+owner_ids[owner_id]+'/history'
    for gameweek in range(1,39):
        lineup_url[owner,gameweek] = 'https://draft.premierleague.com/api/entry/'+owner_ids[owner_id]+'/event/'+str(gameweek)

bootstrap_url = 'https://draft.premierleague.com/api/bootstrap-static'
player_owner_url = 'https://draft.premierleague.com/api/league/5857/element-status'
league_details_url = 'https://draft.premierleague.com/api/league/5857/details'

In [11]:
def url_to_df(url, datatype):
    request = requests.get(url)
    json = request.json()
    df = pd.DataFrame(json[datatype])
    return df

#### Create Dataframes ####

In [149]:
player_types_df = url_to_df(bootstrap_url,'element_types')

teams_df = url_to_df(bootstrap_url,'teams')

player_owner_df = url_to_df(player_owner_url,'element_status')
player_owner_df = player_owner_df.sort_values('element', ignore_index=True)

league_entries_df = url_to_df(league_details_url,'league_entries')
league_standings_df = url_to_df(league_details_url,'standings')
league_standings_df = league_standings_df.sort_values('league_entry', ignore_index=True)
league_standings_df = pd.concat([league_entries_df[['entry_id','id','entry_name','player_first_name','waiver_pick']], league_standings_df[['last_rank','rank','event_total','total']]], axis=1)

player_info_df = url_to_df(bootstrap_url,'elements')
player_info_df['team'] = player_info_df.team.map(teams_df.set_index('id').name)
player_info_df['element_type'] = player_info_df.element_type.map(player_types_df.set_index('id').singular_name)
player_info_df.insert(3,'owner',player_owner_df.owner.map(league_entries_df.set_index('entry_id').player_first_name))

1

In [77]:
team_info_df = {}
team_history_df = {}
for owner in owners:
    team_info_df[owner] = url_to_df(team_details_url[owner],'history')

In [165]:
def get_team_info_by_gameweek(owner, gameweek):
    if gameweek > current_gameweek:
        print("Gameweek not yet played")
        return('ERROR')
    lineup_df = url_to_df(lineup_url[owner, gameweek],'picks')
    team_info_df = lineup_df[['element']]
    team_info_df.insert(1,'name',team_info_df['element'].map(player_info_df.set_index('id').second_name))
    team_info_df.insert(2,'team',team_info_df['element'].map(player_info_df.set_index('id').team))
    team_info_df.insert(3,'position',team_info_df['element'].map(player_info_df.set_index('id').element_type))
    for player in team_info_df['element']:
        player_url = 'https://draft.premierleague.com/api/element-summary/'+str(player)
    return(team_info_df)

In [167]:
get_team_info_by_gameweek('Fat',1)

,element,name,team,position
0,383,Lloris,Spurs,Goalkeeper
1,250,van Dijk,Liverpool,Defender
2,155,Digne,Everton,Defender
3,345,Basham,Sheffield Utd,Defender
4,120,Mount,Chelsea,Midfielder
5,37,Grealish,Aston Villa,Midfielder
6,365,Redmond,Southampton,Midfielder
7,394,Alli,Spurs,Midfielder
8,355,Lundstram,Sheffield Utd,Midfielder
9,460,Jiménez,Wolves,Forward


In [26]:
slim_players_df = pd.DataFrame(player_info_df[['second_name','first_name','team','owner','element_type','minutes','total_points']])


In [27]:
slim_players_df

,second_name,first_name,team,owner,element_type,minutes,total_points
0,Özil,Mesut,Arsenal,NaN,Midfielder,0,0
1,Papastathopoulos,Sokratis,Arsenal,NaN,Defender,0,0
2,Luiz Moreira Marinho,David,Arsenal,NaN,Defender,0,0
3,Aubameyang,Pierre-Emerick,Arsenal,Olivia,Midfielder,90,7
4,Soares,Cédric,Arsenal,NaN,Defender,0,0
...,...,...,...,...,...,...,...
522,Macey,Matt,Arsenal,NaN,Goalkeeper,0,0
523,Woods,Sam,Crystal Palace,NaN,Defender,0,0
524,Odoi,Denis,Fulham,NaN,Defender,90,1
525,Forster,Fraser,Southampton,NaN,Goalkeeper,0,0


In [31]:
slim_players_df.loc[slim_players_df.owner=='Fat']


,second_name,first_name,team,owner,element_type,minutes,total_points
36,Grealish,Jack,Aston Villa,Fat,Midfielder,0,0
73,White,Ben,Brighton,Fat,Defender,0,0
91,Lowton,Matthew,Burnley,Fat,Defender,0,0
119,Mount,Mason,Chelsea,Fat,Midfielder,0,0
154,Digne,Lucas,Everton,Fat,Defender,90,12
249,van Dijk,Virgil,Liverpool,Fat,Defender,90,7
344,Basham,Chris,Sheffield Utd,Fat,Defender,0,0
354,Lundstram,John,Sheffield Utd,Fat,Midfielder,0,0
362,McCarthy,Alex,Southampton,Fat,Goalkeeper,90,2
364,Redmond,Nathan,Southampton,Fat,Midfielder,90,2
